<a href="https://colab.research.google.com/github/mr-cri-spy/SLM/blob/main/Movie_Booking_Bridge.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install fastapi uvicorn openai requests beautifulsoup4


 Installed Necessary Libraries

Creating FastAPI Server

In [2]:
import nest_asyncio
import uvicorn
from fastapi import FastAPI, Request
import openai
import requests

# FastAPI Setup

In [3]:
nest_asyncio.apply()
app = FastAPI()

OPENROUTER_API_KEY = "API"  # Replace with your OpenRouter API key

@app.get("/")
def read_root():
    return {"status": "Crisbee backend running"}

@app.post("/ask")
async def ask_question(request: Request):
    data = await request.json()
    question = data.get("question")

    # Simulated movie data (replace with actual scraping logic later)
    movie_data = """
    - PVR Mysore: Su From So at 7:30PM - 5 seats available in Row F
    - INOX Mall: Su From So at 8:00PM - 4 seats available Row G
    """

    answer = query_llm(question, movie_data)
    return {"answer": answer}

def query_llm(user_query, context_data):
    prompt = f"""
    You are a smart assistant for Mysore movies. Based on the following real-time data:
    {context_data}

    Answer this user question: {user_query}
    """

    headers = {
        "Authorization": f"Bearer {OPENROUTER_API_KEY}",
        "Content-Type": "application/json"
    }

    body = {
        "model": "openai/gpt-3.5-turbo",
        "messages": [{"role": "user", "content": prompt}]
    }

    res = requests.post("https://openrouter.ai/api/v1/chat/completions", headers=headers, json=body)
    return res.json()["choices"][0]["message"]["content"]

# Run the FastAPI app on Colab

In [ ]:
uvicorn.run(app, host="0.0.0.0", port=9001)

INFO:     Started server process [551]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:9001 (Press CTRL+C to quit)


Expose the FastAPI Server using ngrok

In [4]:
!pip install pyngrok


In [ ]:
from pyngrok import ngrok

# Open a ngrok tunnel to the FastAPI app
public_url = ngrok.connect(9001)
print(f"FastAPI is live at: {public_url}")


Test the Backend in Google Colab

In [ ]:
response = requests.post(f"{public_url}/ask", json={"question": "Where can I book 4 seats for Su From So today after  8PM in Mysore?"})
print(response.json())
